In [1]:
from wave_client import WaveClient
import pandas as pd
from utils import (
    load_environment_variables,
)


In [2]:
# Load environment variables and connect to WAVE backend
RESEARCHER_API_KEY, EXPERIMENTEE_API_KEY, WAVE_BACKEND_URL = load_environment_variables()

print(f"🔍 Connecting to WAVE backend...")
print(f"📡 Backend URL: {WAVE_BACKEND_URL}")
print(f"🔑 Using Researcher API key ending in: ...{RESEARCHER_API_KEY[-4:]}")

client = WaveClient(api_key=RESEARCHER_API_KEY, base_url=WAVE_BACKEND_URL)

Present working directory: c:\Users\inkpe\local-JS-Williams\SurprisedElab\tools
Loading environment variables from c:\Users\inkpe\local-JS-Williams\SurprisedElab\tools/.env
🔍 Connecting to WAVE backend...
📡 Backend URL: https://wave-backend-production-8781.up.railway.app
🔑 Using Researcher API key ending in: ...SYuP


In [ ]:
## use this (uncomment it) to see what commands exist 
## (allow the scrollable option at the bottom to see all commands)

# help(client.experiments)
# help(client.experiment_data)

In [3]:
# Get all experiments to see which one to analyze
experiments = await client.experiments.list(skip=0, limit=200)

# Sort by creation date (most recent first) and show up to 5
sorted_experiments = sorted(experiments, key=lambda x: x.get('created_at', ''), reverse=True)[:5]
print(f"Showing {len(sorted_experiments)} most recent experiments (of {len(experiments)} total)\n")

# Display experiment info
for i, exp in enumerate(sorted_experiments, 1):
    print(f"{i}. Experiment UUID: {exp['uuid']}")
    print(f"   Description: {exp.get('description', 'N/A')}")
    print(f"   Type ID: {exp.get('experiment_type_id', 'N/A')}")
    print(f"   Created: {exp.get('created_at', 'N/A')[:10]}")
    print(f"   Tags: {', '.join(exp.get('tags', []))}")
    print("-" * 70)

Showing 5 most recent experiments (of 9 total)

1. Experiment UUID: 4a1d1324-64ab-464f-9787-b19058002f11
   Description: shadows_afc_shortnum_prod
   Type ID: 13
   Created: 2026-02-05
   Tags: 
----------------------------------------------------------------------
2. Experiment UUID: faeb33db-c970-4c8e-9064-00ba7432ab3b
   Description: Test experiment for shadows_afc_shortnum - TEST_shadows_afc_shortnum_20260205_150522
   Type ID: 13
   Created: 2026-02-05
   Tags: test
----------------------------------------------------------------------
3. Experiment UUID: e1e840b2-cfea-46d4-ba60-54707b12bb78
   Description: Test experiment for shadows_afc_shortnum - TEST_shadows_afc_shortnum_20260205_150117
   Type ID: 13
   Created: 2026-02-05
   Tags: test
----------------------------------------------------------------------
4. Experiment UUID: aeb293f8-2206-46a9-9d46-b21092526504
   Description: shadows_afc_numtask_prod
   Type ID: 12
   Created: 2026-02-05
   Tags: 
--------------------------

In [4]:
# Put the necessary info into the variables below 
selected_experiment_uuid = "530a5fd4-2485-4d88-9c83-5acead500b94" 

# Get all data from that experiment
print(f"\n📊 Fetching data for experiment {selected_experiment_uuid}...")
experiment_data_df = await client.experiment_data.get_all_data(
    experiment_id=selected_experiment_uuid
)

print(f"Total records: {len(experiment_data_df)}")
# print(experiment_data_df)



📊 Fetching data for experiment 530a5fd4-2485-4d88-9c83-5acead500b94...
Total records: 854


In [5]:
# Filter and save into .csv file
full_df = (experiment_data_df.query("participant_id != '{{%PROLIFIC_PID%}}'")
           .sort_values(by="timestamp",ascending=True))
full_df.to_csv("data/dataframe_full.csv", index = False)

answer_df = full_df.query("trial_category == 'answerexpt'")
answer_df.to_csv("data/dataframe_answer.csv", index = False)

debrief_df = full_df.query("trial_category == 'debriefexpt'")
debrief_df.to_csv("data/dataframe_debrief.csv", index = False)



In [6]:
# Example of how to pull out only a few specific columns

selected_columns = answer_df[["response","response_time","thisDifference","person_race","person_sex","person_variation"]]
print(selected_columns)

    response  response_time  thisDifference person_race person_sex  \
843       48         8981.0           -26.0           A          M   
837       50        16101.0           -26.0           B          F   
831       50        10206.0            -9.0           L          M   
822       70         5653.0            -5.0           B          M   
821       46         7989.0             4.0           L          F   
..       ...            ...             ...         ...        ...   
25        36        15604.0           -24.0           W          F   
19        34         6764.0           -45.0           A          F   
13        44         8841.0           -20.0           B          F   
7         58        23447.0           -19.0           W          F   
1         63        10783.0           -33.0           L          M   

    person_variation  
843                2  
837                1  
831                1  
822                2  
821                1  
..               ... 